# Dataset importing

In [1]:
#@title Define imports and constants
from google.colab import drive
import os
import time
import math

drive.mount('/content/drive', force_remount=True)

# dataset:
# fbx files (main dataset source)
FBX = '/content/drive/MyDrive/AutoCalibr/dataset/fbx/'
# ply (pre-converted to fix artifacts)
PLY_TO_IMPORT = '/content/drive/MyDrive/AutoCalibr/dataset/ply/'

# intermediate folders
INTERMEDIATES = '/content/intermediates/'
CONVERTED_PLY = '/content/intermediates/converted_ply/'
NORMALIZED_PLY = '/content/intermediates/normalized_ply/'
# NORMALIZED_PLY = '/content/drive/MyDrive/AutoCalibr/normalized_ply/'


# sometimes used for debug outputting into non-cluttered directory
DIR = '/content/'

# these will be automatically determined, do not change them (defined here to allow out-of-order exports)
global_stretch_scale_x = 1
global_stretch_scale_y = 1
global_stretch_scale_z = 1

!rm -r sample_data/ 2>/dev/null
!mkdir {INTERMEDIATES} 2>/dev/null

Mounted at /content/drive


In [ ]:
#@title Convert FBX to PLY {vertical-output: true}

!pip install bpy
!rm -r {CONVERTED_PLY} 2>/dev/null
!mkdir {CONVERTED_PLY}

import bpy

# needs to be rotated counterclockwise (left) 90 degrees - could be mostly automated (noted below)
needs_extra_rotation = ["Ace Of Spades", "Cantata-57", "Chroma Rush", "Cloudstrike", "Dead Mans Tale", "Duality", "False Promises", "Fugue 55", "Hawkmoon", "Jack Queen King 3", "Mindbenders Ambition", "No Time To Explain", "Ruinous Effigy", "Seven Seraph Carbine", "Seventh Seraph CQC-12", "Seventh Seraph Officer Revolver", "Seventh Seraph SAW", "Seventh Seraph SI-2", "Seventh Seraph VY-7", "The Fourth Horseman", "Trustee", "Witherhoard"]

for f in os.listdir(FBX):
  if f.endswith('.fbx'):
    # Isolate the name of the .fbx file (without extension)
    name_no_ext = os.path.splitext(os.path.basename(f))[0]
    print(f"Object: {name_no_ext}\n")

    # Delete all mesh objects to avoid exporting multiple models into the same file
    bpy.ops.object.select_all(action='DESELECT')
    bpy.ops.object.select_by_type(type='MESH')
    bpy.ops.object.delete()

    # Load in FBX file
    bpy.ops.import_scene.fbx(filepath=os.path.join(FBX, f))

    # Select the object
    obj_object = bpy.context.selected_objects[0]
    bpy.context.view_layer.objects.active = obj_object

    # Decide on redirection
    if name_no_ext in needs_extra_rotation:
      forwards_dir = 'Z'
      # Remove the file from the list as it's been accounted for
      needs_extra_rotation.remove(name_no_ext)
    else:
      forwards_dir = '-X'

    # Export object to PLY
    bpy.ops.export_mesh.ply(filepath=os.path.join(CONVERTED_PLY, f.replace('.fbx', '.ply')), use_ascii=True, use_mesh_modifiers=True, use_normals=False, use_uv_coords=False, use_colors=False, axis_forward=forwards_dir, axis_up='Y')

    print('-' * 50)

# Ensure that any models that were supposed to receive extra rotation were hit
if len(needs_extra_rotation) > 0:
  print(f"The following manually-specified models were not hit (check for typos): {needs_extra_rotation}")

Object: Coldheart

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Coldheart.ply' in 0.936
--------------------------------------------------
Object: Vigilance Wing

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Vigilance Wing.ply' in 0.574
--------------------------------------------------
Object: Riskrunner

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Riskrunner.ply' in 0.767
--------------------------------------------------
Object: Skyburner's oath

FBX version: 7400
Export completed "/content/intermediates/converted_ply/Skyburner's oath.ply" in 0.198
--------------------------------------------------
Object: Anarchy

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Anarchy.ply' in 0.840
--------------------------------------------------
Object: The Last Word

FBX version: 7400
Export completed '/content/intermediates/converted_ply/The Last Word.ply' in 0.257
-------------------

# Dataset processing

In [76]:
# @title Define PlyObject Class
from random import choice

class Vertex:
  def __init__(self, x, y, z):
    self.x = x
    self.y = y
    self.z = z


  def scale(self, scale_x, scale_y, scale_z):
    self.x *= scale_x
    self.y *= scale_y
    self.z *= scale_z


  def translate(self, offset_x, offset_y, offset_z):
    self.x += offset_x
    self.y += offset_y
    self.z += offset_z


  def __hash__(self):
    return hash((self.x, self.y, self.z))


  def __eq__(self, other):
    if isinstance(other, Vertex):
      return self.x == other.x and self.y == other.y and self.z == other.z
    return False


class Face:
  def __init__(self, vertices):
    # vertices is an unordered list of the indexes of the connected vertices that form the face
    self.vertices = vertices


  def __lt__(self, other):
    for val1, val2 in zip(self.vertices, other.vertices):
      if val1 < val2:
        return True
      elif val1 > val2:
        return False
    return len(self) < len(other)


  def __hash__(self):
    return hash(tuple(self.vertices))


  def __eq__(self, other):
    if isinstance(other, Face):
      return (sorted(self.vertices) == sorted(other.vertices))
    return False


class PlyObject:
  def __init__(self, name, vertices, faces):
    self.name = name
    self.vertices = vertices
    self.faces = faces


  def save_file(self, filename):
    with open(filename, "w") as file:
      file.write("ply\n")
      file.write("format ascii 1.0\n")
      file.write("comment Created by PlyObject class\n")
      file.write(f"element vertex {len(self.vertices)}\n")
      file.write("property float x\n")
      file.write("property float y\n")
      file.write("property float z\n")
      file.write(f"element face {len(self.faces)}\n")
      file.write("property list uchar uint vertex_indices\n")
      file.write("end_header\n")

      for vertex in self.vertices:
        file.write(f"{vertex.x} {vertex.y} {vertex.z}\n")

      for face in self.faces:
        formatted_vertices = ' '.join(str(v) for v in face.vertices)
        number_of_vertices = len(face.vertices)
        file.write(f"{number_of_vertices} {formatted_vertices}\n")


  def scale(self, scale_x, scale_y, scale_z):
    for vertex in self.vertices:
      vertex.scale(scale_x, scale_y, scale_z)


  def translate(self, offset_x, offset_y, offset_z):
    for vertex in self.vertices:
      vertex.translate(offset_x, offset_y, offset_z)


  def remove_overlapping(self):
    vert_dict = {}
    convert_dict = {}

    new_vertices = []
    # iterate over existing vertices to identify and save unique ones
    for idx, vertex in enumerate(self.vertices):
      if vertex in vert_dict:
        convert_dict[idx] = vert_dict[vertex]
      else:
        # assign a unique index to each vertex
        vert_dict[vertex] = len(new_vertices)
        convert_dict[idx] = len(new_vertices)
        new_vertices.append(vertex)

    # replace original vertices with new, duplicate-free list
    self.vertices = new_vertices

    # apply convert_dict to update face vertices to match new unique indexing
    for face in self.faces:
      face.vertices = [convert_dict[vertex] for vertex in face.vertices]

    # convert faces to set to remove any potential duplicate faces
    self.faces = set(self.faces)


  def add_random_duplicate_vertices(self, count_to_add):
    for _ in range(count_to_add):
      to_duplicate = choice(self.vertices)
      new_vertex = Vertex(to_duplicate.x, to_duplicate.y, to_duplicate.z)
      self.vertices.append(new_vertex)


  def add_random_duplicate_faces(self, count_to_add):
    for _ in range(count_to_add):
      to_duplicate = choice(self.faces)
      new_face = Face(to_duplicate.vertices)
      self.faces.append(new_face)


  def categorize_faces(self):
    face_dict = {}

    for face in obj.faces:
      length = len(face.vertices)

      if length in face_dict:
        face_dict[length] = face_dict[length] + 1
      else:
        face_dict[length] = 1

    return face_dict


  def get_value_extrema(self):
    min_val = float('inf')
    max_val = float('-inf')

    for v in self.vertices:
      min_val = min(min_val, v.x, v.y, v.z)
      max_val = max(max_val, v.x, v.y, v.z)

    return {'min': min_val, 'max': max_val}


  def get_max_values(self):
    max_values = {'x': None, 'y': None, 'z': None}

    for vertex in self.vertices:
      if max_values['x'] is None or vertex.x > max_values['x']:
        max_values['x'] = vertex.x
      if max_values['y'] is None or vertex.y > max_values['y']:
        max_values['y'] = vertex.y
      if max_values['z'] is None or vertex.z > max_values['z']:
        max_values['z'] = vertex.z

    return max_values


  def get_min_values(self):
    min_values = {'x': None, 'y': None, 'z': None}

    for vertex in self.vertices:
      if min_values['x'] is None or vertex.x < min_values['x']:
        min_values['x'] = vertex.x
      if min_values['y'] is None or vertex.y < min_values['y']:
        min_values['y'] = vertex.y
      if min_values['z'] is None or vertex.z < min_values['z']:
        min_values['z'] = vertex.z

    return min_values


  def center_object(self):
    max_vals = self.get_max_values()
    min_vals = self.get_min_values()

    offset_vals = {'x': 0, 'y': 0, 'z': 0}

    for dim in offset_vals:
      center = (max_vals[dim] + min_vals[dim]) / 2
      offset_vals[dim] = -center

    self.translate(offset_vals['x'], offset_vals['y'], offset_vals['z'])


  def normalize_scale(self):
    x_coordinates = [vertex.x for vertex in self.vertices]
    y_coordinates = [vertex.y for vertex in self.vertices]
    z_coordinates = [vertex.z for vertex in self.vertices]

    max_distance = max(x_coordinates + y_coordinates + z_coordinates)
    min_distance = min(x_coordinates + y_coordinates + z_coordinates)
    normalization_range = max_distance - min_distance

    if normalization_range == 0:
      raise ValueError("Normalization range cannot be zero")

    for vertex in self.vertices:
      vertex.x = 2 * (vertex.x - min_distance) / normalization_range - 1
      vertex.y = 2 * (vertex.y - min_distance) / normalization_range - 1
      vertex.z = 2 * (vertex.z - min_distance) / normalization_range - 1


  def squares_to_tris(self):
    new_faces = []
    for face in self.faces:
      if len(face.vertices) == 4:
        new_faces.append(Face([face.vertices[0], face.vertices[1], face.vertices[2]]))
        new_faces.append(Face([face.vertices[0], face.vertices[2], face.vertices[3]]))
      else:
        new_faces.append(face)

    self.faces = new_faces


  def delete_plane(self):
    # Note: only call if the object has a plane artifact!
    max_x_index = max(range(len(self.vertices)), key = lambda index: self.vertices[index].x)
    min_x_index = min(range(len(self.vertices)), key = lambda index: self.vertices[index].x)
    max_z_index = max(range(len(self.vertices)), key = lambda index: self.vertices[index].z)
    min_z_index = min(range(len(self.vertices)), key = lambda index: self.vertices[index].z)

    indices_to_remove = set([max_x_index, min_x_index, max_z_index, min_z_index])

    for f, face in enumerate(self.faces):
      if set(face.vertices).intersection(indices_to_remove) == indices_to_remove:
        del self.faces[f]
        break
    else:
      raise ValueError("Face with all vertices not found")

    for index in sorted(indices_to_remove, reverse=True):
      del self.vertices[index]

    for face in self.faces:
      face.vertices = [idx if idx not in indices_to_remove else -1 for idx in face.vertices]

    self.remove_disconnected_vertices()


  def remove_disconnected_vertices(self):
    connected_vertices = set()
    for face in self.faces:
      connected_vertices |= set(face.vertices)

    shift_indices = []
    new_vertices = []
    for idx, vertex in enumerate(self.vertices):
      if idx in connected_vertices:
        new_vertices.append(vertex)
        shift_indices.append(len(new_vertices) - 1)
      else:
        shift_indices.append(None)

    self.vertices = new_vertices

    for face in self.faces:
      face.vertices = [shift_indices[vertex] if shift_indices[vertex] is not None else None for vertex in face.vertices]
      face.vertices = [vertex for vertex in face.vertices if vertex is not None]

    self.faces = [face for face in self.faces if face.vertices]


  def stretch_to_max(self):
    # note: update scale globals before calling
    self.scale(global_stretch_scale_x, global_stretch_scale_y, global_stretch_scale_z)


  def revert_stretch(self):
    self.scale(1/global_stretch_scale_x, 1/global_stretch_scale_y, 1/global_stretch_scale_z)


  def dimension_range(self):
    xmin = xmax = obj.vertices[0].x
    ymin = ymax = obj.vertices[0].y
    zmin = zmax = obj.vertices[0].z

    for vertex in obj.vertices:
      xmin = min(xmin, vertex.x)
      xmax = max(xmax, vertex.x)
      ymin = min(ymin, vertex.y)
      ymax = max(ymax, vertex.y)
      zmin = min(zmin, vertex.z)
      zmax = max(zmax, vertex.z)

    xrang = xmax - xmin
    yrang = ymax - ymin
    zrang = zmax - zmin

    return {'x':xrang, 'y':yrang, 'z':zrang}


  def sort_tri_data(self):
    index_map = {}

    # sort the vertices by x first, then y, then z
    sorted_vertices = sorted(
      enumerate(self.vertices),
      key=lambda pair: (pair[1].x, pair[1].y, pair[1].z)
    )
    self.vertices = [pair[1] for pair in sorted_vertices]

    # record the new indices of the vertices in the map
    for i, pair in enumerate(sorted_vertices):
      old_index, _ = pair
      index_map[old_index] = i

    # convert old face lists to new face lists using the index map
    new_faces = []
    for face in self.faces:
      new_face_vertices = [index_map[i] for i in face.vertices]
      new_faces.append(Face(new_face_vertices))

    # replace old face list with new face list, which we first sort
    new_faces.sort()

    self.faces = new_faces


  @classmethod
  def from_file(cls, filepath):
    with open(filepath, 'r') as f:
      if next(f).strip() != "ply":
        raise ValueError("The file being read is not a PLY file.")

      for _ in range(2):
        next(f)

      n_vertices = int(next(f).split()[-1])

      for _ in range(3):
        next(f)

      n_faces = int(next(f).split()[-1])

      for _ in range(2):
        next(f)

      vertices = []
      for _ in range(n_vertices):
        x, y, z = map(float, next(f).split())
        vertices.append(Vertex(x, y, z))

      faces = []
      for _ in range(n_faces):
        face_vertices = list(map(int, next(f).split()[1:]))
        faces.append(Face(face_vertices))

    name = os.path.splitext(os.path.basename(filepath))[0]

    return cls(name, vertices, faces)

In [77]:
def add_random_duplicate_vertices(self, count_to_add):
  shifts_dict = {} # This dictionary will keep track of the shifts of vertices indexes

  vertex_indices = list(range(len(self.vertices)))
  for _ in range(count_to_add):
    duplicate_index = random.choice(vertex_indices) # Choose a random vertex to duplicate

    # Insert the duplicate next to the original to preserve the order
    self.vertices.insert(duplicate_index + 1, self.vertices[duplicate_index])

    # Update the shifts_dict to account for the inserted vertex
    for index in range(duplicate_index + 2, len(self.vertices)):
      if index in shifts_dict:
        shifts_dict[index] += 1
      else:
        shifts_dict[index] = 1

  # Update the vertices in the faces with the new indexes
  for face in self.faces:
    face.vertices = [vertex_index + shifts_dict.get(vertex_index, 0) for vertex_index in face.vertices]


In [78]:
#@title Process PLY files as PlyObject {vertical-output: true}

# Toggle between using full dataset and just preconverted subset
use_fbx_dataset = False
use_ply_dataset = False
use_single_test_file = True

ply_objs = []

if use_fbx_dataset:
  for f in os.listdir(CONVERTED_PLY):
    if f.endswith('.ply'):
      start_time = time.time()
      obj = PlyObject.from_file(filepath=os.path.join(CONVERTED_PLY, f))
      ply_objs.append(obj)
      elapsed_time = time.time() - start_time

      print(f"Object: {obj.name}\n")
      print(f'Processed auto-converted PLY into PlyObject (took {elapsed_time:.2f} s)')
      print(f'\nVertice count: {len(obj.vertices)}')
      print(f'Face count: {len(obj.faces)}')
      print('-' * 50)


# Import pre-converted .ply files
if use_ply_dataset:
  for f in os.listdir(PLY_TO_IMPORT):
    if f.endswith('.ply'):
      start_time = time.time()
      obj = PlyObject.from_file(filepath=os.path.join(PLY_TO_IMPORT, f))
      ply_objs.append(obj)
      elapsed_time = time.time() - start_time

      print(f"Object: {obj.name}\n")
      print(f'Processed pre-converted PLY into PlyObject (took {elapsed_time:.2f} s)')
      print(f'\nVertice count: {len(obj.vertices)}')
      print(f'Face count: {len(obj.faces)}')
      print('-' * 50)

f = PLY_TO_IMPORT+'Blind Perdition.ply'
start_time = time.time()
obj = PlyObject.from_file(filepath=os.path.join(PLY_TO_IMPORT, f))
ply_objs.append(obj)
elapsed_time = time.time() - start_time

print(f"Object: {obj.name}\n")
print(f'Processed pre-converted PLY into PlyObject (took {elapsed_time:.2f} s)')
print(f'\nVertice count: {len(obj.vertices)}')
print(f'Face count: {len(obj.faces)}')
print('-' * 50)

Object: Blind Perdition

Processed pre-converted PLY into PlyObject (took 0.18 s)

Vertice count: 13044
Face count: 14070
--------------------------------------------------


In [79]:
#@title Remove plane artifacts (dataset-specific cleaning) {vertical-output: true}

# has a giant rectangular plane originally used as a background, will need to be filtered out
has_plane_artifact = ['Abbadon', 'Blind Perdition', 'Ex Machina', 'Komodo-4FR', 'Nova Mortis', 'Trespasser', 'Vestian Dynasty', 'Vouchsafe', 'Hereafter']

# Remove plane artifact from manually specified objects

# Could also iterate over all objects and use exceptions,
# but would need to give delete_plane function stricter pre-deletion checking
for obj in ply_objs:
  if obj.name in has_plane_artifact:
    has_plane_artifact.remove(obj.name)

    obj_range = obj.dimension_range()

    # Print the absolute maximum and minimum values
    print(f"Object: {obj.name}")
    print("\nDimension Range:")
    print('X:', obj_range['x'])
    print('Y:', obj_range['y'])
    print('Z:', obj_range['z'])

    try:
      start_time = time.time()

      obj.delete_plane()

      elapsed_time = time.time() - start_time

      obj_range = obj.dimension_range()

      print(f"\nPlane detected and deleted (took {elapsed_time:.2f} s)")
      print(f"\nDimension Range (plane deleted in {elapsed_time:.2f} s):")
      print('X:', obj_range['x'])
      print('Y:', obj_range['y'])
      print('Z:', obj_range['z'])
    except ValueError:
      print(f"\nNo plane found!")

    print('-' * 50)

# Ensure that any models that were supposed to receive extra rotation were hit
if len(has_plane_artifact) > 0:
  print(f"\nThe following manually-specified models were not hit (check for typos): {has_plane_artifact}")

Object: Blind Perdition

Dimension Range:
X: 1.831408
Y: 0.47424900000000003
Z: 3.43575

Plane detected and deleted (took 0.09 s)

Dimension Range (plane deleted in 0.09 s):
X: 0.10221
Y: 0.46234
Z: 1.319475
--------------------------------------------------

The following manually-specified models were not hit (check for typos): ['Abbadon', 'Ex Machina', 'Komodo-4FR', 'Nova Mortis', 'Trespasser', 'Vestian Dynasty', 'Vouchsafe', 'Hereafter']


In [80]:
#@title Categorize face data

face_lengths = {}

for obj in ply_objs:
  temp_face_lengths = obj.categorize_faces()

  for length, count in temp_face_lengths.items():
    if length in face_lengths:
      face_lengths[length] += count
    else:
      face_lengths[length] = count

for length, count in face_lengths.items():
  print(f'Faces with {length} vertices: {count} instances')

for length, count in face_lengths.items():
  if length < 3 or length > 4:
    raise Exception(f'\nFace of unsupported size {length}!')

Faces with 3 vertices: 14069 instances


In [81]:
#@title Convert any objects with square faces to tris {vertical-output: true}

face_lengths = {}
square_obj_count = 0
for obj in ply_objs:

  face_data = obj.categorize_faces()

  if 4 in face_data:
    square_obj_count = square_obj_count + 1

    print(f"Object: {obj.name}\n")
    if 3 in face_data:
      print(f"Tris: {face_data[3]}")
    else:
      print(f"Squares: 0")
    print(f"Squares: {face_data[4]}")

    start_time = time.time()

    obj.squares_to_tris()

    elapsed_time = time.time() - start_time

    print(f'\nConverted square faces to tris (took {elapsed_time:.2f} s)\n')
    face_data = obj.categorize_faces()
    print(f"Tris: {face_data[3]}")
    if 4 in face_data:
      print(f"Squares: {face_data[4]}")
    else:
      print(f"Squares: 0")

    print('-' * 50)

if square_obj_count == 0:
  print(f"No objects containing squares found. All objects contain only tris.")

No objects containing squares found. All objects contain only tris.


In [82]:
#@title Merge overlapping vertices and faces {vertical-output: true}

for obj in ply_objs:
    start_time = time.time()

    obj.remove_overlapping()

    elapsed_time = time.time() - start_time

    print(f"Object: {obj.name}\n")

    print(f'Merged any overlaping vertices and faces (took {elapsed_time:.2f} s)')

    print('-' * 50)

Object: Blind Perdition

Merged any overlaping vertices and faces (took 0.05 s)
--------------------------------------------------


In [83]:
#@title Center the objects to the origin {vertical-output: true}

for obj in ply_objs:
    start_time = time.time()

    obj.center_object()

    elapsed_time = time.time() - start_time

    print(f"Object: {obj.name}\n")

    print(f'Centered to origin (took {elapsed_time:.2f} s)')

    print('-' * 50)

Object: Blind Perdition

Centered to origin (took 0.02 s)
--------------------------------------------------


In [84]:
#@title Normalize object scale to perfectly fit boundaries {vertical-output: true}

for obj in ply_objs:

  extrema = obj.get_value_extrema()

  print(f"Object: {obj.name}")
  print("\nExtrema in any Dimension:")
  print(f"Minimum: {extrema['min']}")
  print(f"Maximum: {extrema['max']}")


  start_time = time.time()

  obj.normalize_scale()

  elapsed_time = time.time() - start_time

  print(f'\nNormalised object scale to boundaries (took {elapsed_time:.2f} s)')

  extrema = obj.get_value_extrema()

  print("\nExtrema in any Dimension:")
  print(f"Minimum: {extrema['min']}")
  print(f"Maximum: {extrema['max']}")
  print('-' * 50)

Object: Blind Perdition

Extrema in any Dimension:
Minimum: -0.6597375
Maximum: 0.6597375

Normalised object scale to boundaries (took 0.02 s)

Extrema in any Dimension:
Minimum: -1.0
Maximum: 1.0
--------------------------------------------------


In [85]:
#@title Analyze dimension ranges to pseudo-verify object orientation

# used for checking if I missed any rotation overrides
count_smallest = {'x': 0, 'y': 0, 'z': 0}
count_middle = {'x': 0, 'y': 0, 'z': 0}
count_largest = {'x': 0, 'y': 0, 'z': 0}

for obj in ply_objs:

  obj_range = obj.dimension_range()

  # count the podium placings of ranges for every dimension
  sorted_keys = sorted(obj_range, key=obj_range.get)
  smallest_key = sorted_keys[0]
  middle_key = sorted_keys[1]
  largest_key = sorted_keys[2]

  count_smallest[smallest_key] += 1
  count_middle[middle_key] += 1
  count_largest[largest_key] += 1

print("Times with largest dimensional range:")
print(f"X: {count_largest['x']}")
print(f"Y: {count_largest['y']}")
print(f"Z: {count_largest['z']}")

print("\nTimes with middle dimensional range:")
print(f"X: {count_middle['x']}")
print(f"Y: {count_middle['y']}")
print(f"Z: {count_middle['z']}")

print("\nTimes with smallest dimensional range:")
print(f"X: {count_smallest['x']}")
print(f"Y: {count_smallest['y']}")
print(f"Z: {count_smallest['z']}\n")

Times with largest dimensional range:
X: 0
Y: 0
Z: 1

Times with middle dimensional range:
X: 0
Y: 1
Z: 0

Times with smallest dimensional range:
X: 1
Y: 0
Z: 0



In [86]:
#@title Calculate global stretch values given global dimensional extrema

min_x = min_y = min_z = float('inf')
max_x = max_y = max_z = float('-inf')

for obj in ply_objs:
  for vertex in obj.vertices:
    min_x = min(min_x, vertex.x)
    min_y = min(min_y, vertex.y)
    min_z = min(min_z, vertex.z)
    max_x = max(max_x, vertex.x)
    max_y = max(max_y, vertex.y)
    max_z = max(max_z, vertex.z)

scale_x, scale_y, scale_z = 2/(max_x - min_x), 2/(max_y - min_y), 2/(max_z - min_z)

print(f"Dimensional minima:")
print(f"X: {min_x}")
print(f"Y: {min_y}")
print(f"Z: {min_z}")

print(f"\nDimensional maxima:")
print(f"X: {max_x}")
print(f"Y: {max_y}")
print(f"Z: {max_z}\n")

print('-' * 50)

print(f"\nDerived Stretch Value:")
print(f"X: {scale_x}")
print(f"Y: {scale_y}")
print(f"Z: {scale_z}")

Dimensional minima:
X: -0.07746262718126518
Y: -0.35039693817616857
Z: -1.0

Dimensional maxima:
X: 0.07746262718126529
Y: 0.35039693817616846
Z: 1.0

--------------------------------------------------

Derived Stretch Value:
X: 12.909451130026424
Y: 2.8539062162045252
Z: 1.0


In [87]:
#@title Stretch non-bounded dimensions using constant values to tighten scope {vertical-output: true}

global_stretch_scale_x = scale_x
global_stretch_scale_y = scale_y
global_stretch_scale_z = scale_z

for obj in ply_objs:
  obj_range = obj.dimension_range()

  print(f"Object: {obj.name}")
  print("\nDimension Range:")
  print('X:', obj_range['x'])
  print('Y:', obj_range['y'])
  print('Z:', obj_range['z'])

  start_time = time.time()
  obj.stretch_to_max()
  elapsed_time = time.time() - start_time

  obj_range = obj.dimension_range()

  print(f"\nStretched non-bounded dimensions using global constants (took {elapsed_time:.2f} s):")
  print(f"\nDimension Range (stretched in {elapsed_time:.2f} s):")
  print('X:', obj_range['x'])
  print('Y:', obj_range['y'])
  print('Z:', obj_range['z'])
  print('-' * 50)

Object: Blind Perdition

Dimension Range:
X: 0.15492525436253046
Y: 0.700793876352337
Z: 2.0

Stretched non-bounded dimensions using global constants (took 0.01 s):

Dimension Range (stretched in 0.01 s):
X: 2.0
Y: 2.0
Z: 2.0
--------------------------------------------------


In [88]:
#@title Verify that dataset dimensional extrema are properly constrained

track_min = {'x': float('inf'), 'y': float('inf'), 'z': float('inf')}
track_max = {'x': float('-inf'), 'y': float('-inf'), 'z': float('-inf')}

for obj in ply_objs:

  # Compare and store the min and max for each dimension
  max_values = obj.get_max_values()
  min_values = obj.get_min_values()

  for dimension in ['x', 'y', 'z']:
    track_min[dimension] = min(track_min[dimension], min_values[dimension])
    track_max[dimension] = max(track_max[dimension], max_values[dimension])

print("Dimensional Minima:")
print('X:', track_min['x'])
print('Y:', track_min['y'])
print('Z:', track_min['z'])

print("\nDimensional Maxima:")
print('X:', track_max['x'])
print('Y:', track_max['y'])
print('Z:', track_max['z'])

Dimensional Minima:
X: -0.9999999999999993
Y: -1.0000000000000002
Z: -1.0

Dimensional Maxima:
X: 1.0000000000000007
Y: 0.9999999999999999
Z: 1.0


In [89]:
#@title TODO: Pad with random duplicate vertices {vertical-output: true}

# this will be moved to autoencoder portion once finalized
# will create many (likely 10000+) random duplicates per object

import copy

for obj in ply_objs:
    print(f"Object: {obj.name}\n")
    print(f"Vertex count: {len(obj.vertices)}\n")

    start_time = time.time()

    obj.add_random_duplicate_vertices(21896 - len(obj.vertices))

    elapsed_time = time.time() - start_time

    print(f'Added duplicate vertices as padding (took {elapsed_time:.2f} s)')
    print(f"Vertex count: {len(obj.vertices)}\n")
    print('-' * 50)

Object: Blind Perdition

Vertex count: 7807

Added duplicate vertices as padding (took 0.09 s)
Vertex count: 21896

--------------------------------------------------


In [90]:
#@title Sort order of vertices and faces numerically to remove arbitrary sample noise {vertical-output: true}

for obj in ply_objs:
    start_time = time.time()

    obj.sort_tri_data()

    elapsed_time = time.time() - start_time

    print(f"Object: {obj.name}\n")

    print(f'Sorted object vertices and faces numerically (took {elapsed_time:.2f} s)')

    print('-' * 50)

Object: Blind Perdition

Sorted object vertices and faces numerically (took 0.53 s)
--------------------------------------------------


In [91]:
#@title Export normalized PLY files {vertical-output: true}

!rm -r {NORMALIZED_PLY} 2>/dev/null
!mkdir {NORMALIZED_PLY}

for obj in ply_objs:
  start_time = time.time()
  # todo: remove before final model use
  obj.revert_stretch()
  obj.save_file(NORMALIZED_PLY + obj.name + '.ply')
  elapsed_time = time.time() - start_time


  print(f"Object: {obj.name}\n")
  print(f'Exported normalized PLY file (took {elapsed_time:.2f} s)')
  print('-' * 50)

Object: Blind Perdition

Exported normalized PLY file (took 0.15 s)
--------------------------------------------------


In [92]:
# !rm -f /content/drive/MyDrive/AutoCalibr/normalized_ply/*
# !cp -r /content/intermediates/normalized_ply/ /content/drive/MyDrive/AutoCalibr

# Autoencoder

In [ ]:
#@title Define imports and constants

import ast
import pandas as pd
import warnings
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import pad_sequences

latent_dim = 50

In [ ]:
#@title Import pre-normalized PLY file artifacts(to allow for runtime restart) {vertical-output: true}

ply_objs=[]

for f in os.listdir(NORMALIZED_PLY):
  if f.endswith('.ply'):
    start_time = time.time()
    ply_obj = PlyObject.from_file(filepath=os.path.join(NORMALIZED_PLY, f))
    ply_objs.append(ply_obj)
    elapsed_time = time.time() - start_time

    print(f"Object: {obj.name}\n")
    print(f'Processed pre-normalized PLY file into PlyObject (took {elapsed_time:.2f} s)')
    print(f'\nVertice count: {len(ply_obj.vertices)}')
    print(f'Face count: {len(ply_obj.faces)}')
    print('-' * 50)

In [ ]:
#@title Categorize objects to determine input layer sizes

max_vertices = 0
max_faces = 0

for obj in ply_objs:
  max_vertices = max(max_vertices, len(obj.vertices))
  max_faces = max(max_faces, len(obj.faces))

print("Maximum vertices: ", max_vertices)
print("Vertex neurons (coordinate count): ", max_vertices * 3)

print("\nMaximum faces: ", max_faces)
print("Face neurons (one per using encoding): ", max_faces)

print("\nTotal Input Neurons: ", max_vertices * 3 + max_faces)

In [ ]:
#@title Define variational sampling function

class Sampling(layers.Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.random.normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
#@title Convert PlyObjects to input arrays {vertical-output: true}

# Doing this step before defining encoder and decoder to avoid system memory cap

vertex_inputs_list = []
face_inputs_list = []
for obj in ply_objs:
  vertex_arr = np.array([[vert.x, vert.y, vert.z] for vert in obj.vertices])

  #vertex_arr = pad_sequences(vertex_arr, maxlen=max_vertices, dtype='float32', padding='post')

  vertex_inputs_list.append(vertex_arr)

  print(f"Converted to array: {obj.name}\n")

  obj = None

ply_objs = None

In [ ]:
#@title Initialize encoder model

vertex_inputs = keras.Input(shape=(max_vertices, 3))
face_inputs = keras.Input(shape=(max_faces, 1))

x = layers.Dense(32, activation="relu")(vertex_inputs)
x = layers.Flatten()(x)

y = layers.Dense(32, activation="relu")(face_inputs)
y = layers.Flatten()(y)

x = layers.Concatenate()([x, y])

x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = keras.Model([vertex_inputs, face_inputs], [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
#@title Initialize decoder model

latent_inputs = keras.Input(shape=(latent_dim,))

x = layers.Dense(512, activation="relu")(latent_inputs)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(2048, activation="relu")(x)
x = layers.Dense(max_vertices * 3, activation="relu")(x)

vertex_outputs = layers.Reshape((max_vertices, 3))(x)

y = layers.Dense(512, activation="relu")(latent_inputs)
y = layers.Dense(1024, activation="relu")(y)
y = layers.Dropout(0.5)(y)
y = layers.Dense(2048, activation="relu")(y)
y = layers.Dense(max_faces, activation="relu")(y)

face_outputs = layers.Reshape((max_faces, 1))(y)

decoder = keras.Model(latent_inputs, [vertex_outputs, face_outputs], name="decoder")
decoder.summary()

In [ ]:
#@title Define VAE class

class VAE(keras.Model):
  def __init__(self, encoder, decoder, beta=0.1, **kwargs):
    super().__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.beta = beta
    self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
    self.reconstruction_loss_tracker = keras.metrics.Mean(
      name="reconstruction_loss"
    )
    self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

  @property
  def metrics(self):
    return [
      self.total_loss_tracker,
      self.reconstruction_loss_tracker,
      self.kl_loss_tracker,
    ]

  def train_step(self, data):
    with tf.GradientTape() as tape:
      z_mean, z_log_var, z = self.encoder(data)
      reconstruction = self.decoder(z)
      reconstruction_loss = tf.reduce_mean(
        tf.reduce_sum(
          keras.losses.mse(data, reconstruction), axis=(1, 2)
        )
      )
      kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
      kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
      total_loss = reconstruction_loss + self.beta * kl_loss
    grads = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
    self.total_loss_tracker.update_state(total_loss)
    self.reconstruction_loss_tracker.update_state(reconstruction_loss)
    self.kl_loss_tracker.update_state(kl_loss)
    return {
      "loss": self.total_loss_tracker.result(),
      "reconstruction_loss": self.reconstruction_loss_tracker.result(),
      "kl_loss": self.kl_loss_tracker.result(),
    }

In [ ]:
#@title Train models using defined VAE

'''
vertex_inputs_arr = np.array(vertex_inputs_list)
face_inputs_arr = np.array(face_inputs_list)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit([vertex_inputs_arr, face_inputs_arr], epochs=30, batch_size=128)
'''